In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,2194674.89,-7.42,7.08,-1.88,-1.16,0.41,0.54
4,-7.42,178340.43,-2.38,-0.65,0.21,0.09,-0.01
5,7.08,-2.38,97221.35,-1.32,0.34,0.61,0.04
6,-1.88,-0.65,-1.32,33319.86,0.12,-0.18,0.03
7,-1.16,0.21,0.34,0.12,7129.75,-0.01,0.02
8,0.41,0.09,0.61,-0.18,-0.01,719.71,0.00
9,0.54,-0.01,0.04,0.03,0.02,0.00,62.99


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00001,0.00002,-0.00001,-0.00001,0.00001,0.00005
4,-0.00001,1.00000,-0.00002,-0.00001,0.00001,0.00001,-0.00000
5,0.00002,-0.00002,1.00000,-0.00002,0.00001,0.00007,0.00001
6,-0.00001,-0.00001,-0.00002,1.00000,0.00001,-0.00004,0.00002
7,-0.00001,0.00001,0.00001,0.00001,1.00000,-0.00000,0.00002
8,0.00001,0.00001,0.00007,-0.00004,-0.00000,1.00000,0.00001
9,0.00005,-0.00000,0.00001,0.00002,0.00002,0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

15.424438820575856

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.88254798e+11 4.49517281e+09 1.25182456e+09 1.38105448e+08
 5.62888829e+06 4.33805604e+04 2.93806049e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.999877,-0.013627,-0.007383,-0.002527,-0.000541,-0.000054,-0.000005
1,0.012278,0.988180,-0.148688,-0.034516,-0.007094,-0.000715,-0.000063
2,0.008963,0.144242,0.984019,-0.102293,-0.018772,-0.001876,-0.000165
3,0.003819,0.048238,0.094471,0.991355,-0.076788,-0.007286,-0.000636
4,0.001086,0.013371,0.024588,0.073732,0.995541,-0.051555,-0.004441
5,0.000162,0.001997,0.003658,0.010699,0.050218,0.994829,-0.087533
6,0.000028,0.000352,0.000645,0.001883,0.008799,0.087182,0.996152


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00012344733961744403,
 0.011820419875796073,
 0.015980607704635386,
 0.008645422552948756,
 0.004459267272116985,
 0.005171217233153391,
 0.0038484814690034064]